In [1]:
import glob
import subprocess

- sample_in.txt
- sample_out.txt
- statement.txt
- full_in.txt

In [2]:
# TODO - change this
contest_folder = 'practice'
password = 'w3ak_password_for_practice'

subprocess.run(
    [
        '7z', 'x', f'{contest_folder}.zip',
        '-pw3ak_password_for_practice',
        f'-o{contest_folder}',
        '-y'
    ]
)


7-Zip [64] 17.05 : Copyright (c) 1999-2021 Igor Pavlov : 2017-08-28
p7zip Version 17.05 (locale=utf8,Utf16=on,HugeFiles=on,64 bits,8 CPUs x64)

Scanning the drive for archives:
1 file, 40894256 bytes (39 MiB)

Extracting archive: practice.zip
--
Path = practice.zip
Type = zip
Physical Size = 40894256

Everything is Ok

Files: 23
Size:       84913192
Compressed: 40894256


CompletedProcess(args=['7z', 'x', 'practice.zip', '-pw3ak_password_for_practice', '-opractice', '-y'], returncode=0)

In [3]:
glob.glob("./practice/*/")  # only select folders

['./practice/Line of Delivery (Part 1)/',
 './practice/Line of Delivery (Part 2)/',
 './practice/Line by Line/',
 './practice/Fall in Line/',
 './practice/Walk the Line/']

In [4]:
problem_name = "Line by Line"

In [5]:
with open(f"{contest_folder}/{problem_name}/statement.txt") as f:
    statement = f.read()
    
with open(f"{contest_folder}/{problem_name}/sample_in.txt") as f:
    sample_in = f.read()

with open(f"{contest_folder}/{problem_name}/sample_out.txt") as f:
    sample_out = f.read()
    
with open(f"{contest_folder}/{problem_name}/full_in.txt") as f:
    full_in = f.read()

In [6]:
prompt = f"""
Write Python code to solve this problem

{statement}

This is the sample input

{sample_in}

This is the sample output

{sample_out}

The Python code, when executed, will take standard input and prints to standard output.

Put the Python code within ```python and ```
"""

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(completion.choices[0].message)

In [ ]:
import re

def extract_python_code(response_string):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, response_string, re.DOTALL)
    python_code = '\n\n'.join(matches)
    return python_code

In [ ]:
python_code = extract_python_code(completion.choices[0].message.content)

In [ ]:
import subprocess
import tempfile
import os


def execute_code(code_str, input_str):
    with tempfile.NamedTemporaryFile('w+', suffix='.py', delete=False) as tmp_file:
        tmp_file.write(code_str)
        tmp_file_path = tmp_file.name
    
    try:
        result = subprocess.run(
            ['python', '-u', tmp_file_path],
            input=input_str,
            text=True,
            capture_output=True,
            timeout=5  # Set a timeout to prevent infinite execution
        )
        if result.stderr:
            return result.stdout + "\n\nError:" + result.stderr
        return result.stdout
    
    except subprocess.TimeoutExpired as e:
        return f"{e.stdout.decode() if e.stdout else ''}\nError: Code execution timed out."
    finally:
        os.remove(tmp_file_path)

In [ ]:
sample_executed_output = execute_code(python_code, sample_in)

In [ ]:
full_executed_output = execute_code(python_code, full_in)

In [ ]:
import os

os.makedirs('submission_output', exist_ok=True)
os.makedirs('submission_code', exist_ok=True)

In [ ]:
with open(f"submission_output/{problem_name.replace(' ', '-')}.txt", "w") as f:
    f.write(full_executed_output)

In [ ]:
with open(f"submission_code/{problem_name.replace(' ', '-')}.py", "w") as f:
    f.write(python_code)